In [ ]:
import urllib.request
import json
 
# Your primary key (replace with secondary if you want)
api_key = '72fNK2ogONHXje1XShvxvQ3mZUKZN8kErOTKnZBx1xaEd9apKnoZJQQJ99BFAAAAAAAAAAAAINFRAZML3cS9'
 
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")
 
# Example input data - adjust this according to your model's expected input format
data = {
        "text": "This is a test sentence to evaluate the DeBERTa endpoint."
    }
 
# Encode the data to JSON bytes
body = str.encode(json.dumps(data))
 
# Your endpoint URL
url = 'http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score'
 
# Set the request headers including authorization with your API key
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + api_key
}
 
# Prepare the request
req = urllib.request.Request(url, body, headers)
 
try:
    # Send the request and read the response
    response = urllib.request.urlopen(req)
    result = response.read()
    print("Response from DeBERTa endpoint:")
    response = eval(result.decode("utf-8"))
    print(response)
 
except urllib.error.HTTPError as error:
    # If there's an HTTP error, print status and details
    print("The request failed with status code:", error.code)
    print("Headers:", error.info())
    print("Error response:", error.read().decode("utf8", 'ignore'))

Response from DeBERTa endpoint:
"{\"predictions\": {\"emotion\": \"unknown\", \"sub_emotion\": \"unknown\", \"intensity\": \"unknown\"}, \"raw_predictions\": {\"emotion\": [[-2.3535149097442627, -2.7075352668762207, -2.1163432598114014, 2.768012762069702, 3.4213383197784424, -2.740438461303711, 1.3640868663787842]], \"sub_emotion\": [[-0.908567488193512, 0.7767903208732605, -4.667848587036133, -0.5953039526939392, -0.41957715153694153, -0.660086452960968, 1.8608988523483276, 4.128921985626221, -2.0373587608337402, -1.554884910583496, -1.1481451988220215, -3.3077480792999268, -3.654724597930908, -0.7455893754959106, -2.2528023719787598, -4.40749454498291, -5.066255569458008, -2.410703182220459, -4.219160079956055, -0.38470709323883057, 4.7725701332092285, 0.5356895327568054, -0.6146451234817505, 2.2942633628845215, 0.11964304745197296, -4.125598907470703, -2.3472158908843994, -0.8247337937355042]], \"intensity\": [[3.7233965396881104, 1.0488967895507812, -3.915595293045044]]}, \"confide

In [28]:
# Example: Complete workflow for getting final predictions
def get_final_predictions(api_response):
    """
    Complete workflow to extract and decode predictions from API response.
    
    Args:
        api_response (dict): Full response from the emotion classification API
        
    Returns:
        dict: Final decoded predictions with metadata
    """
    if api_response.get('status') != 'success':
        return {'error': 'API call was not successful'}
    
    # Extract raw predictions
    raw_predictions = api_response.get('raw_predictions', {})
    
    # Decode predictions
    final_predictions = decode_predictions(raw_predictions)
    
    if final_predictions:
        # Add metadata
        final_predictions['status'] = 'success'
        final_predictions['input_text'] = "This is a test sentence to evaluate the DeBERTa endpoint."
        final_predictions['model_info'] = {
            'endpoint': 'deberta-emotion-clf-endpoint',
            'prediction_classes': {
                'emotions_available': len(emotion_encoder.classes_),
                'sub_emotions_available': len(sub_emotion_encoder.classes_),
                'intensities_available': len(intensity_encoder.classes_)
            }
        }
    
    return final_predictions

# Get the final prediction dictionary
pred_d = get_final_predictions(response)

emotion, sub_emotion, intensity = pred_d.get("emotion"), pred_d.get("sub_emotion"), pred_d.get("intensity")

print("Emotion: ", emotion, 
      "\nSub-emotion: ", sub_emotion, 
      "\nIntensity: ", intensity)

Emotion:  neutral 
Sub-emotion:  melancholy 
Intensity:  mild


In [29]:
response

{'predictions': {'emotion': 'unknown',
  'sub_emotion': 'unknown',
  'intensity': 'unknown'},
 'raw_predictions': {'emotion': [[-2.3535149097442627,
    -2.7075352668762207,
    -2.1163432598114014,
    2.768012762069702,
    3.4213383197784424,
    -2.740438461303711,
    1.3640868663787842]],
  'sub_emotion': [[-0.908567488193512,
    0.7767903208732605,
    -4.667848587036133,
    -0.5953039526939392,
    -0.41957715153694153,
    -0.660086452960968,
    1.8608988523483276,
    4.128921985626221,
    -2.0373587608337402,
    -1.554884910583496,
    -1.1481451988220215,
    -3.3077480792999268,
    -3.654724597930908,
    -0.7455893754959106,
    -2.2528023719787598,
    -4.40749454498291,
    -5.066255569458008,
    -2.410703182220459,
    -4.219160079956055,
    -0.38470709323883057,
    4.7725701332092285,
    0.5356895327568054,
    -0.6146451234817505,
    2.2942633628845215,
    0.11964304745197296,
    -4.125598907470703,
    -2.3472158908843994,
    -0.8247337937355042]],
  '

In [ ]:
import urllib.request
import json
import pickle
import numpy as np

class AzureEmotionPredictor:
    """
    Handles emotion prediction via Azure endpoint, decoding, and post-processing.
    Ensures sub-emotion is valid for the predicted main emotion.
    """

    def __init__(self, api_key, endpoint_url, encoders_dir="models/encoders"):
        """
        Initialize with API key, endpoint URL, and encoder directory.
        """
        self.api_key = api_key
        self.endpoint_url = endpoint_url
        self.encoders_dir = encoders_dir
        self.emotion_mapping = {
            "curiosity": "happiness",
            "neutral": "neutral",
            "annoyance": "anger",
            "confusion": "surprise",
            "disappointment": "sadness",
            "excitement": "happiness",
            "surprise": "surprise",
            "realization": "surprise",
            "desire": "happiness",
            "approval": "happiness",
            "disapproval": "disgust",
            "embarrassment": "fear",
            "admiration": "happiness",
            "anger": "anger",
            "optimism": "happiness",
            "sadness": "sadness",
            "joy": "happiness",
            "fear": "fear",
            "remorse": "sadness",
            "gratitude": "happiness",
            "disgust": "disgust",
            "love": "happiness",
            "relief": "happiness",
            "grief": "sadness",
            "amusement": "happiness",
            "caring": "happiness",
            "nervousness": "fear",
            "pride": "happiness",
        }
        self._load_encoders()

    def _load_encoders(self):
        """
        Load label encoders for emotion, sub_emotion, and intensity.
        """
        try:
            with open(f"{self.encoders_dir}/emotion_encoder.pkl", "rb") as f:
                self.emotion_encoder = pickle.load(f)
            with open(f"{self.encoders_dir}/sub_emotion_encoder.pkl", "rb") as f:
                self.sub_emotion_encoder = pickle.load(f)
            with open(f"{self.encoders_dir}/intensity_encoder.pkl", "rb") as f:
                self.intensity_encoder = pickle.load(f)
        except Exception as e:
            raise RuntimeError(f"Failed to load encoders: {e}")

    def get_prediction(self, text):
        """
        Send a request to the Azure endpoint and return the raw response.
        """
        data = {"text": text}
        body = str.encode(json.dumps(data))
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'Authorization': 'Bearer ' + self.api_key
        }
        req = urllib.request.Request(self.endpoint_url, body, headers)
        try:
            response = urllib.request.urlopen(req)
            result = response.read()
            return json.loads(result.decode("utf-8"))
        except urllib.error.HTTPError as error:
            raise RuntimeError(f"Request failed: {error.code} {error.read().decode('utf8', 'ignore')}")

    def decode_and_postprocess(self, raw_predictions):
        """
        Decode raw predictions and post-process sub-emotion to ensure consistency.
        """
        # Defensive: handle both logits and index predictions
        try:
            # If predictions are logits, take argmax
            if isinstance(raw_predictions.get("emotion"), (list, np.ndarray)):
                emotion_idx = int(np.argmax(raw_predictions["emotion"]))
            else:
                emotion_idx = int(raw_predictions["emotion"])
            if isinstance(raw_predictions.get("sub_emotion"), (list, np.ndarray)):
                sub_emotion_logits = np.array(raw_predictions["sub_emotion"])
                # Will post-process below
            else:
                sub_emotion_logits = None
                sub_emotion_idx = int(raw_predictions["sub_emotion"])
            if isinstance(raw_predictions.get("intensity"), (list, np.ndarray)):
                intensity_idx = int(np.argmax(raw_predictions["intensity"]))
            else:
                intensity_idx = int(raw_predictions["intensity"])
        except Exception as e:
            raise ValueError(f"Malformed raw predictions: {e}")

        # Decode main emotion
        emotion = self.emotion_encoder.inverse_transform([emotion_idx])[0]

        # Post-process sub-emotion: only allow sub-emotions that map to the predicted emotion
        if sub_emotion_logits is not None:
            sub_emotion_classes = self.sub_emotion_encoder.classes_
            # Get valid sub-emotion indices for this emotion
            valid_indices = [
                i for i, label in enumerate(sub_emotion_classes)
                if self.emotion_mapping.get(label) == emotion
            ]
            if valid_indices:
                # Pick the valid sub-emotion with highest logit
                best_idx = valid_indices[np.argmax(sub_emotion_logits[valid_indices])]
                sub_emotion = sub_emotion_classes[best_idx]
            else:
                # Fallback: pick the most probable sub-emotion
                sub_emotion = self.sub_emotion_encoder.inverse_transform([int(np.argmax(sub_emotion_logits))])[0]
        else:
            # If only index is given, just decode
            sub_emotion = self.sub_emotion_encoder.inverse_transform([sub_emotion_idx])[0]
            # Optionally, check mapping and fallback if not valid
            if self.emotion_mapping.get(sub_emotion) != emotion:
                # Fallback: pick first valid sub-emotion for this emotion
                sub_emotion_classes = self.sub_emotion_encoder.classes_
                valid_labels = [label for label in sub_emotion_classes if self.emotion_mapping.get(label) == emotion]
                if valid_labels:
                    sub_emotion = valid_labels[0]

        # Decode intensity
        intensity = self.intensity_encoder.inverse_transform([intensity_idx])[0]

        return {
            "emotion": emotion,
            "sub_emotion": sub_emotion,
            "intensity": intensity
        }

    def predict(self, text):
        """
        Full workflow: get prediction, decode, and post-process.
        """
        api_response = self.get_prediction(text)
        if api_response.get('status') != 'success':
            raise RuntimeError("API call was not successful")
        raw_predictions = api_response.get('raw_predictions', {})
        return self.decode_and_postprocess(raw_predictions)

# Example usage:
# predictor = AzureEmotionPredictor(api_key, endpoint_url, encoders_dir="models/encoders")
# result = predictor.predict("This is a test sentence to evaluate the DeBERTa endpoint.")
# print(result)